In [21]:
import pycurl
import json
import pandas as pd
from io import BytesIO 
from IPython.core.display import display, HTML

In [22]:
# Config Model Endpoint 
# E.g. MODEL_ENDPOINT = 'https://hidden-star-7888.pub.playground.bdrk.ai'
MODEL_ENDPOINT = 'https://rock-star-3910.pub.sandbox-aws.bdrk.ai'
MODEL_INFER_ENDPOINT = MODEL_ENDPOINT + "/infer"
MODEL_METRICS_ENDPOINT = MODEL_ENDPOINT + "/metrics" # Bedrock-specific. Do not change!

## Ping Endpoint using Pycurl

In [23]:
def ping_indexpage(url):
    '''Ping index page at /'''
    b_obj = BytesIO() 
    crl = pycurl.Curl() 

    # Set URL value
    crl.setopt(crl.URL, url)
    # Write bytes that are utf-8 encoded
    crl.setopt(crl.WRITEDATA, b_obj)
    # Perform the request
    crl.perform() 
    # End curl session
    crl.close()
    # Get the content stored in the BytesIO object (in byte characters) 
    get_body = b_obj.getvalue()
    # Decode the bytes stored in get_body to HTML and print the result 
    # print('Output of GET request:\n%s' % get_body.decode('utf8')) 
    display(HTML(get_body.decode('utf8')))
    
ping_indexpage(MODEL_ENDPOINT)

## Testing the model inference

### 1. Prepare Test Data

In [24]:
import requests
import pandas as pd

def get_row_of_data(df, row_index=0):
    row = df.iloc[row_index]    
    data = {}
    for feat in FEATURES:
        data[feat] = row[feat]
        
    print("TARGET == {}".format(row["TARGET"]))

    return data

In [25]:
df = pd.read_csv("data/creditdata/creditdata_test_v2.csv")
df.head(1)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,TARGET
0,1559,50000.0,2,3,1,31,1,2,2,2,...,40906.0,42100.0,41269.0,2830.0,1000.0,1800.0,2000.0,0.0,1704.0,0


In [26]:
FEATURES = [
#     'ID',
    'LIMIT_BAL',
    'SEX',
    'EDUCATION',
    'MARRIAGE',
    'AGE',
    'PAY_1',
    'PAY_2',
    'PAY_3',
    'PAY_4',
    'PAY_5',
    'PAY_6',
    'BILL_AMT1',
    'BILL_AMT2',
    'BILL_AMT3',
    'BILL_AMT4',
    'BILL_AMT5',
    'BILL_AMT6',
    'PAY_AMT1',
    'PAY_AMT2',
    'PAY_AMT3',
    'PAY_AMT4',
    'PAY_AMT5',
    'PAY_AMT6'
]

### 2. Send a POST request to Model Endpoint

In [27]:
# Load data
data = get_row_of_data(df, 0)

# Configure headers
headers = {
    'Content-Type': 'application/json',
    'X-Bedrock-Api-Token': ''
}

response = requests.post(MODEL_INFER_ENDPOINT, headers=headers, data=json.dumps(data))
print(response.ok)

TARGET == 0.0
True


In [28]:
MODEL_INFER_ENDPOINT

'https://rock-star-3910.pub.sandbox-aws.bdrk.ai/infer'

In [29]:
print(response.text)

{"predicted_proba":0.24}



In [30]:
print(response.json())

{'predicted_proba': 0.24}
